In [4]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Logistic Regression

There are many options for regression. Recall that **linear regression** is helpful when predicting a real number along some continuum without a restricted range. Things like income and age.

But what about . . .
* probabilities
* binary values (sick vs. healthy, male vs. female)
* approval rating (0% to 100%)

We need some "squishification" process to map numbers from a real number continuum to the unit interval (X range $-\infty$ to $\infty$, Y range 0 to 1). We need a **cumulative density function** - most commonly the **[sigmoid function](https://en.wikipedia.org/wiki/Sigmoid_function)**. This function in particular is useful because its calculous properties give it nice behaviors. (e^x derivative is e^x) Another common CDF is the **[probit function](https://en.wikipedia.org/wiki/Probit)** for probabilities.

With logistic regression, we can even model general multinomial classification problems by combining several logistic regressions to predict membership in various classes and outputting the class that is most likely.

At its heart, logistic regression is still linear. That is, the underlying math and optimization follow traditional linear regression. However, our coefficients won't have intuitive linear interpretation.

$S(x) = \frac{1}{1 + e^{-x}} = \frac{e^x}{e^x + 1}$

## Resources

[5 Reasons “Logistic Regression” should be the first thing you learn when becoming a Data Scientist](https://towardsdatascience.com/5-reasons-logistic-regression-should-be-the-first-thing-you-learn-when-become-a-data-scientist-fcaae46605c4)


**What data science methods are used at work?**
[![](img/kaggle-common-algos.PNG)](https://www.kaggle.com/surveys/2017)

Logistic regression is the baseline for classification models, as well as a handy way to predict probabilities (since those too live in the unit interval). While relatively simple, it is also the foundation for more sophisticated classification techniques such as neural networks (many of which can effectively be thought of as networks of logistic models).

## Example 1: Linear vs. Logistic, Return of the Titanic 🚢

To pull Titanic data from Kaggle:
* create account
* generate API Key and `kaggle.json`
* join [Titanic competition](https://www.kaggle.com/c/titanic/data)

The [Kaggle API](https://github.com/Kaggle/kaggle-api) enables lots of convenient methods for interacting with Kaggle.

Here's how the data were sourced:
```python
!pip install kaggle

import os, json

path_to_config = "../kaggle.json"
kaggle_config = json.loads(open(path_to_config).read())
os.environ['KAGGLE_USERNAME'] = kaggle_config['username']
os.environ['KAGGLE_KEY'] = kaggle_config['key']

!kaggle competitions download -c titanic -p datasets/titanic
```

In [14]:
# How would we try to do this with linear regression?
train_df = pd.read_csv('datasets/titanic/train.csv').dropna()
test_df = pd.read_csv('datasets/titanic/train.csv').dropna()  # Unlabeled, for Kaggle submission

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S


In [34]:
predictors = ['Pclass', 'Age', 'Fare']
target = 'Survived'

X = train_df[predictors]
y = train_df[target]

linear_reg = LinearRegression().fit(X, y)
print('score:', linear_reg.score(X, y))

linear_reg.predict(test_df[predictors])[:10]

score: 0.08389810726550939


array([0.66567686, 0.68168731, 0.52351404, 0.74909452, 0.4779952 ,
       0.58445856, 0.73115482, 0.91675714, 0.57710857, 0.43722395])

How can you "partially" survive? You either survived or did not, like how our target column is encoded.

In [25]:
{attr: coef for attr, coef in zip(predictors, linear_reg.coef_)}

{'Age': -0.008293140798696792,
 'Fare': 0.00048775407963190954,
 'Pclass': -0.08596294986392504}

In [26]:
test_case = np.array([[1, 5, 500]])  # Rich 5-year old in first class
linear_reg.predict(test_case)

array([1.14845883])

How can survial be greater than 1?

In [33]:
log_reg = LogisticRegression().fit(X, y)
print(log_reg.score(X, y))

log_reg.predict(test_df[predictors])[:10]

0.7103825136612022


array([1, 1, 1, 1, 0, 1, 1, 1, 1, 0], dtype=int64)

In [35]:
log_reg.predict(test_case)[0]

1

In [37]:
log_reg.predict_proba(test_case)[0]

array([0.02485552, 0.97514448])

In [59]:
# What's the math?
{attr: coef for attr, coef in zip(predictors, log_reg.coef_[0])}

{'Age': -0.02912512677284113,
 'Fare': 0.004803700867242946,
 'Pclass': -0.04550170440859573}

In [39]:
log_reg.intercept_

array([1.45878264])

In [0]:
# The logistic sigmoid "squishing" function, implemented to accept numpy arrays
def sigmoid(x):
    return 1 / (1 + np.e**(-x))

In [17]:
sigmoid(log_reg.intercept_ + np.dot(log_reg.coef_, np.transpose(test_case)))

array([[0.97514448]])

So, clearly a more appropriate model in this situation! For more on the math, [see this Wikipedia example](https://en.wikipedia.org/wiki/Logistic_regression#Probability_of_passing_an_exam_versus_hours_of_study).

**Note:** while the sign (-/+) of coefficients provides information on the relationship between X and Y, we can't infer that a one-unit difference in X corresponds to a one-unit difference in Y.

## Example 2: Multinomial Logistic Regression

[Absenteeism at work dataset](http://archive.ics.uci.edu/ml/datasets/Absenteeism+at+work) has 21 classes. `sklearn.linear_model.LogisticRegression` automatically handles more than two classes by essentially treating each label as different (1) from some base class (0).

Here's how the data were sourced:


In [106]:
data_url = ('http://archive.ics.uci.edu/ml/'
            'machine-learning-databases/00445/'
            'Absenteeism_at_work_AAA.zip')

extract_zip_url(data_url, 'datasets/abs')

In [107]:
df = pd.read_csv('datasets/abs/Absenteeism_at_work.csv', sep=';')
pd.options.display.max_columns = df.shape[1]
df.head()

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,Hit target,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239.554,97,0,1,2,1,0,1,90,172,30,4
1,36,0,7,3,1,118,13,18,50,239.554,97,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,239.554,97,0,1,0,1,0,0,89,170,31,2
3,7,7,7,5,1,279,5,14,39,239.554,97,0,1,2,1,1,0,68,168,24,4
4,11,23,7,5,1,289,36,13,33,239.554,97,0,1,2,1,0,1,90,172,30,2


In [ ]:
def multinom_logistic(df, target, predictors, test_size=0.5, random_state=42):
    X = df[predictors]
    y = df[target]

    if test_size:
        X_train, X_test, Y_train, Y_test = (
            train_test_split(X, y, test_size=test_size, 
                             random_state=random_state)
        )
    else:
        X_train, X_test = [X] * 2
        Y_train, Y_test = [y] * 2
        
    model = LogisticRegression(random_state=random_state, solver='lbfgs', 
                               multi_class='multinomial', max_iter=5000)

    model.fit(X_train, Y_train)
    print('score:', model.score(X_test, Y_test))
    
    return model

In [82]:
target = 'Reason for absence'
predictors = sorted(list(set(df.columns) - 
                         set(['ID', 'Reason for absence'])))

0.5121621621621621


In [100]:
# without train-test
model = multinom_logistic(df, target, predictors, test_size=None, random_state=42)

score: 0.5121621621621621


In [93]:
model = multinom_logistic(df, target, predictors, test_size=0.5, random_state=42)

score: 0.3810810810810811


In [96]:
# smaller test size makes worse score
model = multinom_logistic(df, target, predictors, test_size=0.2, random_state=42)

score: 0.3783783783783784


## Example 3: real-world classification

We're going to check out a larger dataset - the [FMA Free Music Archive data](https://github.com/mdeff/fma). It has a selection of CSVs with metadata and calculated audio features that you can load and try to use to classify genre of tracks. To get you started:

- Clean up the variable names in the dataframe
- Use logistic regression to fit a model predicting (primary/top) genre
- Inspect, iterate, and improve your model
- Answer the following questions (written, ~paragraph each):
  - What are the best predictors of genre?
  - What information isn't very useful for predicting genre?
  - What surprised you the most about your results?

In [3]:
# # downloads 1.36 GB
# data_url = 'https://os.unil.cloud.switch.ch/fma/fma_metadata.zip'
# extract_zip_url(data_url, 'datasets/fma', flatten=True)

In [141]:
pd.options.display.max_columns = None

In [150]:
tracks = (pd.read_csv('datasets/fma/tracks.csv', header=1)
            .rename(columns={'Unnamed: 0':'track_id'})
            .drop(0, axis=0))

C:\Users\City_Year\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [151]:
tracks.head()

,track_id,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,tags,title,tracks,type,active_year_begin,active_year_end,associated_labels,bio,comments.1,date_created.1,favorites.1,id.1,latitude,location,longitude,members,name,related_projects,tags.1,website,wikipedia_page,split,subset,bit_rate,comments.2,composer,date_created.2,date_recorded,duration,favorites.2,genre_top,genres,genres_all,information.1,interest,language_code,license,listens.1,lyricist,number,publisher,tags.2,title.1
1,2,0.0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4.0,1.0,<p></p>,6073.0,NaN,[],AWOL - A Way Of Life,7.0,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0.0,2008-11-26 01:42:32,9.0,1.0,40.058324,New Jersey,-74.405661,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,small,256000.0,0.0,NaN,2008-11-26 01:48:12,2008-11-26 00:00:00,168.0,2.0,Hip-Hop,[21],[21],NaN,4656.0,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293.0,NaN,3.0,NaN,[],Food
2,3,0.0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4.0,1.0,<p></p>,6073.0,NaN,[],AWOL - A Way Of Life,7.0,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0.0,2008-11-26 01:42:32,9.0,1.0,40.058324,New Jersey,-74.405661,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,medium,256000.0,0.0,NaN,2008-11-26 01:48:14,2008-11-26 00:00:00,237.0,1.0,Hip-Hop,[21],[21],NaN,1470.0,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514.0,NaN,4.0,NaN,[],Electric Ave
3,5,0.0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4.0,1.0,<p></p>,6073.0,NaN,[],AWOL - A Way Of Life,7.0,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0.0,2008-11-26 01:42:32,9.0,1.0,40.058324,New Jersey,-74.405661,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,small,256000.0,0.0,NaN,2008-11-26 01:48:20,2008-11-26 00:00:00,206.0,6.0,Hip-Hop,[21],[21],NaN,1933.0,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151.0,NaN,6.0,NaN,[],This World
4,10,0.0,2008-11-26 01:45:08,2008-02-06 00:00:00,NaN,4.0,6.0,NaN,47632.0,NaN,[],Constant Hitmaker,2.0,Album,NaN,NaN,"Mexican Summer, Richie Records, Woodsist, Skul...","<p><span style=""font-family:Verdana, Geneva, A...",3.0,2008-11-26 01:42:55,74.0,6.0,NaN,NaN,NaN,"Kurt Vile, the Violators",Kurt Vile,NaN,"['philly', 'kurt vile']",http://kurtvile.com,NaN,training,small,192000.0,0.0,Kurt Vile,2008-11-25 17:49:06,2008-11-26 00:00:00,161.0,178.0,Pop,[10],[10],NaN,54881.0,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135.0,NaN,1.0,NaN,[],Freeway
5,20,0.0,2008-11-26 01:45:05,2009-01-06 00:00:00,NaN,2.0,4.0,"<p> ""spiritual songs"" from Nicky Cook</p>",2710.0,NaN,[],Niris,13.0,Album,1990-01-01 00:00:00,2011-01-01 00:00:00,NaN,<p>Songs written by: Nicky Cook</p>\n<p>VOCALS...,2.0,2008-11-26 01:42:52,10.0,4.0,51.895927,Colchester England,0.891874,Nicky Cook\n,Nicky Cook,NaN,"['instrumentals', 'experimental pop', 'post pu...",NaN,NaN,training,large,256000.0,0.0,NaN,2008-11-26 01:48:56,2008-01-01 00:00:00,311.0,0.0,NaN,"[76, 103]","[17, 10, 76, 103]",NaN,978.0,en,Attribution-NonCommercial-NoDerivatives (aka M...,361.0,NaN,3.0,NaN,[],Spiritual Level


In [152]:
tracks.shape

(106574, 53)

In [156]:
tracks.isna().sum()

track_id                  0
comments                  0
date_created           3529
date_released         36280
engineer              91279
favorites                 0
id                        0
information           23425
listens                   0
producer              88514
tags                      0
title                  1025
tracks                    0
type                   6508
active_year_begin     83863
active_year_end      101199
associated_labels     92303
bio                   35418
comments.1                0
date_created.1          856
favorites.1               0
id.1                      0
latitude              62030
location              36364
longitude             62030
members               59725
name                      0
related_projects      93422
tags.1                    0
website               27318
wikipedia_page       100993
split                     0
subset                    0
bit_rate                  0
comments.2                0
composer            

In [ ]:
target = 'genre_top'
predictors = ''

This is the biggest data you've played with so far, and while it does generally fit in Colab, it can take awhile to run. That's part of the challenge!

Your tasks:


*Important caveats*:
- This is going to be difficult data to work with - don't let the perfect be the enemy of the good!
- Be creative in cleaning it up - if the best way you know how to do it is download it locally and edit as a spreadsheet, that's OK!
- If the data size becomes problematic, consider sampling/subsetting
- You do not need perfect or complete results - just something plausible that runs, and that supports the reasoning in your written answers

If you find that fitting a model to classify *all* genres isn't very good, it's totally OK to limit to the most frequent genres, or perhaps trying to combine or cluster genres as a preprocessing step. Even then, there will be limits to how good a model can be with just this metadata - if you really want to train an effective genre classifier, you'll have to involve the other data (see stretch goals).

This is real data - there is no "one correct answer", so you can take this in a variety of directions. Just make sure to support your findings, and feel free to share them as well! This is meant to be practice for dealing with other "messy" data, a common task in data science.

## Resources and stretch goals

- Check out the other .csv files from the FMA dataset, and see if you can join them or otherwise fit interesting models with them
- [Logistic regression from scratch in numpy](https://blog.goodaudience.com/logistic-regression-from-scratch-in-numpy-5841c09e425f) - if you want to dig in a bit more to both the code and math (also takes a gradient descent approach, introducing the logistic loss function)
- Create a visualization to show predictions of your model - ideally show a confidence interval based on error!
- Check out and compare classification models from scikit-learn, such as [SVM](https://scikit-learn.org/stable/modules/svm.html#classification), [decision trees](https://scikit-learn.org/stable/modules/tree.html#classification), and [naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html). The underlying math will vary significantly, but the API (how you write the code) and interpretation will actually be fairly similar.
- Sign up for [Kaggle](https://kaggle.com), and find a competition to try logistic regression with
- (Not logistic regression related) If you enjoyed the assignment, you may want to read up on [music informatics](https://en.wikipedia.org/wiki/Music_informatics), which is how those audio features were actually calculated. The FMA includes the actual raw audio, so (while this is more of a longterm project than a stretch goal, and won't fit in Colab) if you'd like you can check those out and see what sort of deeper analysis you can do.